In [1]:
#base
import pandas as pd
import numpy as np
import re
from itertools import product
from collections import Counter

#scikit-learn modules
from sklearn import ensemble, linear_model, metrics
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict,train_test_split
from sklearn.utils import resample
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

#plotting
from matplotlib import pyplot as plt
from matplotlib import style
style.use('ggplot')

In [6]:
#amino acid position for sparse encoding
aa_pos = {"A":0,"C":1,"D":2,"E":3,"F":4,"G":5,"H":6,"I":7,"K":8,"L":9,"M":10,"N":11,"P":12,"Q":13,"R":14,"S":15,"T":16,"V":17,"W":18,"Y":19,"-":20}
#blosum62 substitution matrix
aa_blosum62 = {"A":np.asarray([4,0,-2,-1,-2,0,-2,-1,-1,-1,-1,-2,-1,-1,-1,1,0,0,-3,-2,-100]),
          "C":np.asarray([0,9,-3,-4,-2,-3,-3,-1,-3,-1,-1,-3,-3,-3,-3,-1,-1,-1,-2,-2,-100]),
          "D":np.asarray([-2,-3,6,2,-3,-1,-1,-3,-1,-4,-3,1,-1,0,-2,0,-1,-3,-4,-3,-100]),
          "E":np.asarray([-1,-4,-2,5,-3,-2,0,-3,1,-3,-2,0,-1,2,0,0,-1,-2,-3,-2,-100]),
          "F":np.asarray([-2,-2,-3,-3,6,-3,-1,0,-3,0,0,-3,-4,-3,-3,-2,-2,-1,1,3,-100]),
          "G":np.asarray([0,-3,-1,-2,-3,6,-2,-4,-2,-4,-3,0,-2,-2,-2,0,-2,-3,-2,-3,-100]),
          "H":np.asarray([-2,-3,-1,0,-1,-2,8,-3,-1,-3,-2,1,-2,0,0,-1,-2,-3,-2,2,-100]),
          "I":np.asarray([-1,-1,-3,-3,0,-4,-3,4,-3,2,1,-3,-3,-3,-3,-2,-1,3,-3,-1,-100]),
          "K":np.asarray([-1,-3,-1,-1,-3,-2,-1,-3,5,-2,-1,0,-1,1,2,0,-1,-2,-3,-2,-100]),
          "L":np.asarray([-1,-1,-4,-3,0,-4,-3,2,-2,4,2,-3,-3,-2,-2,-2,-1,1,-2,-1,-100]),
          "M":np.asarray([-1,-1,-3,-2,0,-3,-2,1,-1,2,5,-2,-2,0,-1,-1,-1,1,-1,-1,-100]),
          "N":np.asarray([-2,-3,1,0,-3,0,1,-3,0,-3,-2,6,-2,0,0,1,0,-3,-4,-2,-100]),
          "P":np.asarray([-1,-3,-1,-1,-4,-2,-2,-3,-1,-3,-2,-2,7,-1,-2,-1,-1,-2,-4,-3,-100]),
          "Q":np.asarray([-1,-3,-0,2,-3,-2,0,-3,1,-2,0,0,-1,5,1,0,-1,-2,-2,-1,-100]),
          "R":np.asarray([-1,-3,-2,0,-3,-2,0,-3,2,-2,-1,0,-2,1,5,-1,-1,-3,-3,-2,-100]),
          "S":np.asarray([1,-1,0,0,-2,0,-1,-2,0,-2,-1,-1,-1,0,-1,4,1,-2,-3,-2,-100]),
          "T":np.asarray([0,-1,-1,-1,-2,-2,-2,-1,-1,-1,-1,0,-1,-1,-1,1,5,0,-2,-2,100]),
          "V":np.asarray([0,-1,-3,-2,-1,-3,-3,3,-2,1,1,-3,-2,-2,-3,-2,0,4,-3,-1,-100]),
          "W":np.asarray([-3,-2,-4,-3,1,-2,-2,-3,-3,-2,-1,-4,-4,-2,-3,-3,-2,-3,11,2,-100]),
          "Y":np.asarray([-2,-2,-3,-2,3,-3,2,-1,-2,-1,-1,-2,-3,-1,-2,-2,-2,-1,2,7,-100]),
          "-":np.asarray([-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,100,-100,-100,0])}
#amino acid combination for 3-mer encoding
aa = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
combinations_list = ["".join(comb) for comb in product(aa, repeat=3)]

#sparse encoding: for each amino acid there is a vector of 20 elements in which the only 1 is the one corresponding to the amino acid in that position, the rest are 0s
def sparse_peptide(peptide_list): 
    pept_vector = np.zeros((len(peptide_list),100,21,1))
    counter = 0
    counter_list = 0
    for peptide in peptide_list:
        if len(peptide) > 100:
            peptide = peptide[0:100]
        for i in range(len(peptide)):
            pept_vector[counter_list,i,aa_pos[peptide[i]]]=1
        for j in range(len(peptide),100):
            pept_vector[counter_list,j,aa_pos["-"]]=1
        counter_list +=1
    lista = np.zeros((len(peptide_list),2100))
    for i in range(len(pept_vector)):
        lista[i] = pept_vector[i].flatten()
    return lista

#denser encoding where for each position of a vector of lenght 100 there is one number representing a specific aa
def denser_peptide(peptide_list):
    pept_vector = np.zeros((len(peptide_list),100))
    counter_list = 0
    for peptide in peptide_list:
        if len(peptide) > 100:
            peptide = peptide[0:100]
        for i in range(len(peptide)):
            pept_vector[counter_list,i]=aa_pos[peptide[i]]
        counter_list +=1
    return pept_vector

#for each position of the peptide amino acid there is a vector of 21 elements with the substitution scores of the blosum62 matrix
def blosum_substitution(peptide_list):
    pept_vector = np.zeros((len(peptide_list),100,21))
    counter = 0
    counter_list = 0
    for peptide in peptide_list:
        if len(peptide) > 100:
            peptide = peptide[0:100]
        for i in range(len(peptide)):
            pept_vector[counter_list,i]=aa_blosum62[peptide[i]]
            counter += 1
        for j in range(counter,100):
            pept_vector[counter_list,j]=aa_blosum62["-"]
        counter_list +=1
        counter = 0
    lista = np.zeros((len(peptide_list),2100))
    for i in range(len(pept_vector)):
        lista[i] = pept_vector[i].flatten()
    return lista

#for each peptide the 3mers are calculated with a sliding window and their number are added to a vector with all the combinations
def threemers(peptide_list):
    pept_vector = np.zeros((len(peptide_list),8000))
    counter_list = 0
    for peptide in peptide_list:
        if len(peptide) > 100:
            peptide = peptide[0:100]
        counts = {aa_comb: 0 for aa_comb in combinations_list}
        for i in range(len(peptide)-2):
            window = peptide[i:i+3]
            if window in counts:
                counts[window] += 1
        pept_vector[counter_list]=list(counts.values())
        counter_list +=1
    return pept_vector

peptides_df = pd.read_csv("DB/DB_hyperfiltered.csv") #load the file containing the peptide sequences
funzioni = peptides_df.Function.unique() #get the list of classes in the database
vectors = [sparse_peptide, dender_peptide, blosum_substitution, threemers] #put all the encoding in a list to call them later
logistic_regression = linear_model.LogisticRegression() #create the classifiers
random_forest = ensemble.RandomForestClassifier()
kneighbor = neighbors.KNeighborsClassifier()
support_vector_machine = svm.SVC()
classifiers = [logistic_regression, random_forest, kneighbor, support_vector_machine] #put all classifiers in a list to call later
#some dictionaries to store stats of training
logreg = {"sparse_peptide":{},"denser_peptide":{}, "blosum_substitution":{}, "threemers":{}}
randfor = {"sparse_peptide":{},"denser_peptide":{}, "blosum_substitution":{}, "threemers":{}}
kneigh = {"sparse_peptide":{},"denser_peptide":{}, "blosum_substitution":{}, "threemers":{}}
suppvem = {"sparse_peptide":{},"denser_peptide":{}, "blosum_substitution":{}, "threemers":{}}

classifier_dict = [logreg,randfor,kneigh,suppvem]
for c in classifier_dict:
    for i in c:
        for j in funzioni:
            c[i][j]=[]


In [7]:
#for each classifier specify the parameters to search in grid search
linear_GS = {"penalty" : ["l1", "l2", "elasticnet", None], "C" : [0.1, 1, 10, 100], "solver" : ["lbfgs", "liblinear", "saga"]}
randomf_GS = {"n_estimators":[10,50,100,200], "criterion":["gini", "entropy", "log_loss"], "max_features":["sqrt", "log2", None] }
kneigh_GS = {"n_neighbors":[2,5,10,20,50], "weights":["uniform", "distance"],"algorithm" : ["auto", "ball_tree", "kd_tree", "brute"],"leaf_size" : [10,30,50]}
svm_GS = {"C" : [0.1, 1, 10, 100],"kernel":["linear", "poly", "rbf", "sigmoid"], "gamma":["scale", "auto"], "class_weight" : ["balanced"]}
GS_parameters = [linear_GS,randomf_GS,kneigh_GS,svm_GS]

In [8]:
#GRID SEARCH
#each classifier method is tested using the 4 different encoding methods on each class and results are printed
grid_results = [] #dict to store results
for clf in range(len(classifiers)): #for each classifier
    print("Using the following classifier:\n",classifiers[clf],"\n")
    for j in vectors: #for each encoding function
        print("\nUsing the following encoding:\n",j.__name__,"\n")
        for i in funzioni: #for each functional class
            print("Analyzing the following function:\n",i)
            peptides_df = pd.read_csv("DB/DB_hyperfiltered.csv") #read all peptides from DB
            peptides_df.loc[peptides_df["Function"] == i, "Class"] = 0 #set the class to test to have label = 0
            functions = peptides_df["Class"] #get all the unique functional classes
            all_data = j(peptides_df["Sequence"]) #encode the data besed on the encoding method
            X_train, X_test, y_train, y_test = model_selection.train_test_split(all_data,functions, stratify=functions, test_size=0.3) #split train and test sets(70/30)
            grid = GridSearchCV(classifiers[clf], GS_parameters[clf], refit = True, scoring="matthews_corrcoef") #grid search using MCC as scoring function
            grid.fit(X_train, y_train)
            print(grid.best_params_)
            i=np.where(grid.cv_results_["rank_test_score"] == 1) #get the best set of hyperparameters
            grid.cv_results_["rank_test_score"][i[0][0]] 
            grid_results.append((grid.best_params_,grid.cv_results_["mean_test_score"][i[0][0]]))
print(grid_results)

Using the following classifier:
 SVC() 


Using the following encoding:
 blosum_substitution 

Analyzing the following function:
 Antidiabetic
{'C': 100, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
Analyzing the following function:
 Antihypertensive
{'C': 100, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'poly'}
Analyzing the following function:
 Antimicrobial
{'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'sigmoid'}
Analyzing the following function:
 Antioxidant
{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
Analyzing the following function:
 Cardiovascular
{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
Analyzing the following function:
 Celiac disease
{'C': 0.1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear'}
Analyzing the following function:
 Immunomodulatory
{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
Analyzing the following function:
 Neuro